In [ ]:
%%capture
!pip install llama-index==0.10.22 llama-index-embeddings-openai qdrant-client llama-index-vector-stores-qdrant

In [ ]:
import os
import nest_asyncio
from getpass import getpass
nest_asyncio.apply()

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = getpass("Enter your Qdrant API Key:")

# Query Pipelines

LlamaIndex offers a query API for chaining modules to manage data workflows easily. It revolves around the QueryPipeline, where you link various modules like LLMs, prompts, and retrievers in a sequence or DAG for end-to-end execution.

You can streamline workflows efficiently using QueryPipeline, reducing code complexity and enhancing readability. Additionally, a declarative interface ensures easy serialization of pipeline components for portability and deployment across systems in the future.

In [ ]:
from llama_index.core.settings import Settings
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.fastembed import FastEmbedEmbedding

Settings.llm = Cohere(model="command-r")
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [102]:
from qdrant_client import QdrantClient
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# Create a Qdrant client
client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

# Create a Qdrant vector store
vector_store = QdrantVectorStore(client=client, collection_name="it_can_be_done")

# Create a vector store index
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=Settings.embed_model,
)


In [103]:
query_engine = index.as_query_engine(similarity_top_k=7, return_sources=True)

In [64]:
query_engine.get_prompts()

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x1180315a0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYS

In [104]:
from llama_index.core.query_pipeline import QueryPipeline

p = QueryPipeline(
    chain=[Settings.llm, query_engine], verbose=True
)

In [105]:
resp = p.run("What advice from the text about Forget the past and move forward with strength and courage")

> Running module 3196d0d2-d565-435b-afc3-a181c9eeb74e with input: 
messages: What advice from the text about Forget the past and move forward with strength and courage

> Running module ee46ba93-3398-4ab4-ac37-caf283066a3a with input: 
input: assistant: The advice from the text about forgetting the past and moving forward with strength and courage is to let go of any negative experiences or regrets that may be holding you back. It encourag...



In [106]:
print(resp)

The text advises to leave behind past failures and disappointments, focusing on the present and future with a positive outlook. It emphasizes the importance of resilience, courage, and determination in overcoming obstacles and moving forward confidently. By releasing negative experiences and regrets, one can free themselves from emotional burdens and approach challenges with strength and optimism, fostering personal growth and a brighter future.


In [78]:
q = QueryPipeline(verbose=True)

In [79]:
q.add_modules({
    "llm": Settings.llm,
    "query_engine": query_engine,
})

In [81]:
q.add_link("llm", "query_engine")

In [88]:
resp = q.run("What Poems aligns with themes in Stoic Philosophy?")

> Running module llm with input: 
messages: What Poems aligns with themes in Stoic Philosophy?

> Running module query_engine with input: 
input: assistant: One poem that aligns with themes in Stoic Philosophy is "Invictus" by William Ernest Henley. This poem reflects the Stoic idea of inner strength and resilience in the face of adversity. The...



In [90]:
resp.__dict__

{'response': 'The poem "Invictus" by William Ernest Henley aligns with themes in Stoic Philosophy by emphasizing inner strength and resilience in the face of adversity. The speaker in the poem declares their unconquerable soul despite the challenges and hardships they have endured, showcasing a steadfast determination to persevere. This reflects the Stoic belief in accepting and overcoming obstacles with courage and fortitude.',
 'source_nodes': [NodeWithScore(node=TextNode(id_='092ba663-a734-4eea-bdfb-a746c62e100a', embedding=None, metadata={'file_path': '../data/it_can_be_done.txt', 'file_name': 'it_can_be_done.txt', 'file_type': 'text/plain', 'file_size': 405217, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'

In [101]:
dir(resp.source_nodes[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo

In [93]:
resp.source_nodes[1].__dict__

{'node': TextNode(id_='1af5fd6e-f19f-413a-8092-6c16365ff39b', embedding=None, metadata={'file_path': '../data/it_can_be_done.txt', 'file_name': 'it_can_be_done.txt', 'file_type': 'text/plain', 'file_size': 405217, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='../data/it_can_be_done.txt', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../data/it_can_be_done.txt', 'file_name': 'it_can_be_done.txt', 'file_type': 'text/plain', 'file_size': 405217, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-21'}, hash='3e88a2e65bd25814a1bc7195033b970f47d5e51fc92783ca4d40918848e8de0c'), <NodeRelationship.PREVIOUS: '2'>: Related

<iframe
    width="100%"
    height="600px"
    src="rag_dag.html"
    frameborder="0"
    allowfullscreen
></iframe>



<iframe
    width="100%"
    height="600px"
    src="rag_dag.html"
    frameborder="0"
    allowfullscreen
    
></iframe>
